In [2]:
# from predset_revised import MahjongSLpredset
from dataset import MahjongSLDataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import torch
from torch import nn

# from model import predEntryModule
from collections import defaultdict

# from model_batched_MNN import MahJongNetBatchedRevised, TileProbModuleNew
import feature
import matplotlib.pyplot as plt
import json
import rule

# from fanCalcLib import formMinComb_c
import copy


def default_zero():
    return 0


def default_list():
    return []


def default_one():
    return 1


def default_three_zeros():
    return [0, 0, 0]


plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["figure.figsize"] = (15, 7)
plt.rcParams["font.size"] = 16

# fmt:off
tile_list_raw = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9',  # 饼
                 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9',  # 万
                 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9',  # 条
                 'F1', 'F2', 'F3', 'F4', 'J1', 'J2', 'J3'  # 风、箭
                 ]

tile_list_raw_cn = ["一饼","二饼","三饼","四饼","五饼","六饼","七饼","八饼","九饼",  # 饼
                 "一万","二万","三万","四万","五万","六万","七万","八万","九万",  # 万
                 "一条", "二条", "三条", "四条", "五条", "六条", "七条", "八条", "九条",   # 条
                 "东风", "南风","西风","北风", '中', '发', '白'  # 风、箭
                 ]
fan_list_raw = ["大四喜", "大三元", "绿一色", "九莲宝灯", "四杠", "连七对", "十三幺", "清幺九", "小四喜", "小三元", "字一色", "四暗刻", "一色双龙会",
                "一色四同顺", "一色四节高", "一色四步高", "三杠", "混幺九", "七对", "七星不靠", "全双刻", "清一色", "一色三同顺", "一色三节高", "全大",
                "全中", "全小", "清龙", "三色双龙会", "一色三步高", "全带五", "三同刻", "三暗刻", "全不靠", "组合龙", "大于五", "小于五", "三风刻", "花龙",
                "推不倒", "三色三同顺", "三色三节高", "无番和", "妙手回春", "海底捞月", "杠上开花", "抢杠和",
                "碰碰和", "混一色", "三色三步高", "五门齐", "全求人", "双暗杠", "双箭刻", "全带幺",
                "不求人", "双明杠", "和绝张", "箭刻", "圈风刻", "门风刻", "门前清", "平和", "四归一",
                "双同刻", "双暗刻", "暗杠", "断幺", "一般高", "喜相逢", "连六", "老少副", "幺九刻",
                "明杠", "缺一门", "无字", "边张", "嵌张", "单钓将", "自摸"]
# fmt:on

In [2]:
path_to_file = "../neg_slim.json"
with open(path_to_file, "r") as f:
    out = json.load(f)
out = out["negative_dict"]
label = out["label"]
pred = out["pred"]
info = out["info"]
data = out["data"]
out.keys()

dict_keys(['data', 'pred', 'label', 'info', 'negative_action_acc', 'negative_action_total', 'negative_tile_acc', 'negative_tile_total', 'acc', 'total'])

In [5]:
out["total"]

58

In [3]:
for id in range(len(label)):
    if len(label[id]) > 5:
        print(id)
        break

1


In [6]:
# transmission file
# uncomment all to run

dict_of_file = defaultdict(default_zero)
for i in range(1):
    dict_of_file[info[i][0]] += 1
file_no_list = list(dict_of_file.keys())
for file_no in file_no_list:
    os.system(
        "scp llf@162.105.31.94:/media/storage/llf/MahJong_STD/sl_prep_revised6_QQR/{}.npy ../tmp/".format(
            file_no
        )
    )

In [9]:
# load data
error_id = 1
(
    meta_feature_new,
    tile_wall_feature,
    tile_prep,
    fan_prep,
    missing_tile_prep,
    count_prep,
    chi_peng_count_remain,
) = data[error_id]
action_mapping = ["Pass", "Chi-1", "Chi-2", "Chi-3", "Peng"]
file_no, line_id, player_id = info[error_id]
observation = "Tile Throw"
if len(label[error_id]) <= 5:
    observation = "Action"
    truth_observation = action_mapping[np.argmax(label[error_id])]
    truth_observation_val = pred[error_id][np.argmax(label[error_id])]
    predicted_observation = action_mapping[np.argmax(pred[error_id])]
    predicted_observation_val = pred[error_id][np.argmax(pred[error_id])]
else:
    truth_observation = tile_list_raw_cn[np.argmax(label[error_id])]
    truth_observation_val = pred[error_id][np.argmax(label[error_id])]
    predicted_observation = tile_list_raw_cn[np.argmax(pred[error_id])]
    predicted_observation_val = pred[error_id][np.argmax(pred[error_id])]

idx_pair = [(pred[error_id][i], i) for i in range(34)]
sorted_idx_pair = sorted(idx_pair, key=lambda x: -x[0])

player_seat = [
    "东",
    "南",
    "西",
    "北",
]
file = "{}.npy".format(file_no)
(
    botzone_log,  # 1
    tileWall_log,  # 2
    pack_log,  # 3
    handWall_log,  # 4
    obsWall_log,  # 5
    remaining_tile_log,  # 6
    botzone_id,  # 7
    winner_id,  # 8
    prevailing_wind,  # 9
    fan_sum,  # 10
    score,
    fan_list,  # 11
) = feature.load_log("../data", file)
print("https://botzone.org.cn/match/{}".format(botzone_id))
print("Observe Turn {} for seat {}".format(line_id - 3, player_seat[player_id]))
print(
    "Observe {} With Truth: {}({:.2f}); Predicted: {}({:.2f})".format(
        observation,
        truth_observation,
        truth_observation_val,
        predicted_observation,
        predicted_observation_val,
    )
)
print(
    "Top 3: {}: {:.2f}, {}: {:.2f}, {}: {:.2f}".format(
        tile_list_raw[sorted_idx_pair[0][1]],
        sorted_idx_pair[0][0],
        tile_list_raw[sorted_idx_pair[1][1]],
        sorted_idx_pair[1][0],
        tile_list_raw[sorted_idx_pair[2][1]],
        sorted_idx_pair[2][0],
    )
)
print("botzone_log: {}".format(botzone_log[line_id : line_id + 2]))

https://botzone.org.cn/match/6160348e5ddc087351c05dc3
Observe Turn 3 for seat 东
Observe Tile Throw With Truth: 西风(3.20); Predicted: 四条(3.90)
Top 3: T4: 3.90, J3: 3.77, J1: 3.54
botzone_log: ['Player 0 Play F3\n' 'Player 1 Draw W4\n']


In [13]:
(
    meta_feature_new,
    tile_wall_feature,
    tile_prep,
    fan_prep,
    missing_tile_prep,
    count_prep,
    chi_peng_count_remain,
) = (
    torch.tensor(meta_feature_new).view(-1, 6),
    torch.tensor(tile_wall_feature).view(-1, 34, 1, 5),
    torch.tensor(tile_prep).view(-1, 5, 64, 34, 7),
    torch.tensor(fan_prep).view(-1, 5, 64, 80),
    torch.tensor(missing_tile_prep).view(-1, 5, 64, 34),
    torch.tensor(count_prep).view(-1, 5, 64, 2),
    torch.tensor(chi_peng_count_remain).view(-1, 5, 64),
)

In [14]:
tile_prep.shape

torch.Size([1, 5, 64, 34, 7])

In [25]:
class TileProbModuleNew(nn.Module):
    """
    Computation for opponent_desire_prob, held_prob, discard_prob
    """

    def __init__(self):
        super(TileProbModuleNew, self).__init__()
        self.cnn_channel = 4
        self.linear_output = 4
        self.tile_nn = nn.Sequential(
            nn.Conv1d(1, self.cnn_channel, 3, stride=1),
            nn.MaxPool1d(3),
        )

        self.game_length_modifier = nn.Sequential(
            # nn.Linear(6, 16),
            # nn.Hardswish(),
            # nn.Linear(16, self.linear_output),
            nn.Linear(6, self.linear_output),
        )

        self.pooling = nn.Sequential(
            nn.MaxPool1d(self.linear_output),
        )

        self.final_layer = nn.Sequential(
            nn.Linear(self.cnn_channel, 1),
        )

    def forward(self, meta_feature, tile_wall_feature):
        # input: meta (data[:4])
        tile_wall_feature_flatten = tile_wall_feature.view(-1, 1, 5)
        tw_computed = self.tile_nn(tile_wall_feature_flatten)
        tw_computed = tw_computed.view(-1, 34, self.cnn_channel, 1)
        length_computed = self.game_length_modifier(meta_feature).view(
            -1, 1, 1, self.linear_output
        )
        mixture = tw_computed * length_computed
        mixture = mixture.view(-1, self.cnn_channel, self.linear_output)
        mixture_maxed = self.pooling(mixture)
        mixture_maxed = mixture_maxed.squeeze()
        final = self.final_layer(mixture_maxed)
        return final.view(-1, 34)


def reversed_tile_conversion(list_rep, key_order_list):
    ret_dict = defaultdict(default_zero)
    for i in range(len(key_order_list)):
        ret_dict[key_order_list[i]] += list_rep[i]
    return ret_dict


class MahJongNetBatchedRevised(nn.Module):
    """
    This module computes data_prep ([32*4, 80, 32, 2]) object
    """

    def __init__(self, device="cpu"):
        super(MahJongNetBatchedRevised, self).__init__()
        self.device = device
        self.fan_coeff = nn.Parameter(
            torch.tensor(
                [0.1 + torch.randn(1) * 0.015 for _ in range(80)],
                requires_grad=True,
            )
        )
        self.fan_coeff_multiplier = torch.tensor(
            [1.0 for _ in range(80)], device=device
        )
        self.fan_coeff_multiplier[33] *= 21
        self.tile_coeff = nn.Parameter(
            torch.tensor(
                [1.0 + torch.randn(1) * 0.015 for _ in range(34)], requires_grad=True
            )
        )
        self.QQR_pack_penalty = nn.Parameter(torch.tensor(0.1, requires_grad=True))

        self.prob_module_throw = TileProbModuleNew().to(device)

    def access_named_params(self):
        return (
            self.fan_coeff.detach().numpy(),
            self.tile_coeff.detach().numpy(),
            self.QQR_pack_penalty.item(),
        )

    def forward(self, x):
        """
        @data_entry: n * 5 * 32 * (34*4, 80, 34, 2)
        @held_prob: n * 5 * 32 * (34)
        @discard_prob: n * 5 * 32 * (34)
        """

        (meta_feature, tile_wall_feature, search_matrix) = x

        prob_throw = self.prob_module_throw(meta_feature, tile_wall_feature)

        (
            tile_prep_data,
            fan_prep,
            missing_tile_prep_data,
            count_prep,
            chi_peng_count_remain_data,
        ) = search_matrix

        missing_tile_prep = missing_tile_prep_data.view(-1, 320, 34)
        chi_peng_count_remain = chi_peng_count_remain_data.view(-1, 320)
        tile_prep = tile_prep_data.view(-1, 320, 34, 7)
        tile_self_draw_baseline = tile_prep[:, :, :, 0]
        tile_need_count = tile_prep[:, :, :, 1]
        tile_cp_coeff = tile_prep[:, :, :, 2]
        tile_cp_base = tile_prep[:, :, :, 3]
        tile_hidden_ct = tile_prep[:, :, :, 4]
        tile_cp_switch = tile_prep[:, :, :, 5]
        tile_null_filler = tile_prep[:, :, :, 6]

        fan_prep_prime = 1 - fan_prep

        # Throw operation
        # fan preference
        fan_pref = fan_prep * self.fan_coeff * self.fan_coeff_multiplier
        multiplied_fan_pref_throw = torch.sum(fan_pref, dim=-1).view(
            -1, 320
        )  # n * 5 * 64

        # tile probability (throw)
        term1_throw = (
            tile_self_draw_baseline * prob_throw.unsqueeze(1) ** tile_need_count
        )
        term2_throw = tile_cp_base * tile_cp_coeff / tile_hidden_ct  #
        term2_throw_zeroed = torch.nan_to_num(term2_throw)
        term3_throw = tile_null_filler
        terms_summed_throw = (
            term1_throw + term3_throw + term2_throw_zeroed * tile_cp_switch
        )

        multiplied_sum_throw = (
            torch.prod(terms_summed_throw, -1) * 100 * multiplied_fan_pref_throw
        )
        final_prob_throw = (
            multiplied_sum_throw * self.QQR_pack_penalty**chi_peng_count_remain
        )

        # distinguish by missing tile (Throw)
        missing_tile_prob_throw = missing_tile_prep * final_prob_throw.unsqueeze(-1)
        # missing_tile_prob_throw += tile_pref_from_fan

        # combine 64 sub entries
        # win-rate transfered to tile, judge tile by tile-bounded win-rate
        missing_tile_prob_throw = missing_tile_prob_throw.view(-1, 5, 64, 34)
        tile_bound_winrate_throw = missing_tile_prob_throw.sum(dim=-2)

        # final tile-bounded prob
        weighted_tile_prob = (tile_bound_winrate_throw * self.tile_coeff)[:, 0]

        # combine 64 sub entries
        # reshape
        # final_prob: win rate for each 64 entries
        final_prob_ops = final_prob_throw.view(-1, 5, 64)
        # win-rate transfered to tile, judge tile by tile-bounded win-rate
        missing_tile_prob_ops = missing_tile_prob_throw
        tile_bound_winrate_ops = missing_tile_prob_ops.sum(dim=2)
        final_prob_ops = final_prob_ops.sum(dim=-1)

        # produce result
        # final tile-bounded prob
        max_tile_prob_for_action, e = torch.max(tile_bound_winrate_ops, dim=2)
        max_tile_prob_for_action[:, 0] = final_prob_ops[:, 0]
        # max_tile_prob_for_action *= self.chi_peng_coeff

        print(
            missing_tile_prob_throw.shape,
            fan_prep.shape,
            missing_tile_prep.shape,
            tile_need_count.shape,
        )

        return (
            missing_tile_prob_throw[0, 0],
            fan_prep,
            missing_tile_prep.view(5, 64, 34),
            tile_need_count.view(5, 64, 34),
        )

        # final tile-bounded prob
        weighted_tile_prob = tile_bound_winrate_throw  # * self.tile_coeff  # - opponent_desire_prob.unsqueeze(1)

        return weighted_tile_prob[:, 0, :]

In [26]:
device = "cpu"
net = MahJongNetBatchedRevised(device).to(device)
net.load_state_dict(
    torch.load(
        "../log/07_01_19_29_35-model_slim_revD/checkpoint/best_acc.pkl",
        map_location=torch.device("cpu"),
    )
)

<All keys matched successfully>

In [27]:
explosion, fan, throw, missing = net(
    (
        meta_feature_new.view(-1, 6),
        tile_wall_feature.view(-1, 34, 1, 5),
        (tile_prep, fan_prep, missing_tile_prep, count_prep, chi_peng_count_remain),
    )
)
explosion *= 1 / torch.max(explosion)

torch.Size([1, 5, 64, 34]) torch.Size([1, 5, 64, 80]) torch.Size([1, 320, 34]) torch.Size([1, 320, 34])


In [29]:
# Generate pandas dataframe
out = explosion.detach().numpy()
df = pd.DataFrame(out, columns=tile_list_raw_cn)
df.style.highlight_max(axis=0)
df = df.round(3)
# Fan list
fan_list = []
for sample in fan[0, 0]:
    fan_list.append([fan_list_raw[i] for i in range(80) if sample[i] == 1])
df["fan"] = fan_list
# Redundant/useless tile
throw_list = []
for sample in throw[0]:
    throw_list.append([tile_list_raw[i] for i in range(34) if sample[i] == 1])
df["discard_choice"] = throw_list
# Tiles in need
tile_list = []
for sample in missing[0]:
    tmp_list = []
    for i in range(34):
        for _ in range(int(sample[i])):
            tmp_list.append(tile_list_raw[i])
    tile_list.append(tmp_list)
df["missing"] = tile_list
# target
target_list = []
for i in range(64):
    hand_list = []
    if len(tile_list[i]) != 0 and len(throw_list[i]) != 0:
        hand_dict = handWall_log[line_id - 1][player_id]
        hand_list = [k for k in hand_dict.keys() for _ in range(hand_dict[k])]
        for t in tile_list[i]:
            hand_list.append(t)
        for t in throw_list[i]:
            hand_list.remove(t)
    target_list.append(copy.deepcopy(hand_list))
df["target"] = target_list

In [30]:
for key in tile_list_raw_cn:
    if sum(df[key]) == 0:
        df.drop(key, axis=1, inplace=True)

In [31]:
df = df.style.set_properties(
    subset=truth_observation, **{"background-color": "green"}
).set_properties(
    subset=predicted_observation, **{"background-color": "red", "color": "white"}
)

In [32]:
s = ["<HTML>"]
s.append("<HEAD><TITLE>WorkSheet</TITLE></HEAD>")
s.append("<BODY>")
# s.append(df.to_html())
s.append(df.to_html())
s.append("</BODY></HTML>")
html = "".join(s)
html_file = open("neg{}.html".format(error_id), "w")
html_file.write(html)
html_file.close()